In [ ]:
from scipy import signal, linalg
import scipy.io.wavfile
import numpy as np
import os
import math
import sys

path_db = 'm.conte/05_AudioClassifier_Pdf/05_AudioClassifier_Pdf/database/'
path_music = path_db + 'music/'
path_speech = path_db + 'speech/'
path_unknowns = path_db.replace('database/', '') + 'unknownSounds/'
print(os.getcwd() + path_music)

spg_music = 0
spg_speech = 0


def load_audio(path):
    audio_list = []  #in py questa è una semplice lista vuota
    for filename in os.listdir(path):
        audio_list.append(
            (scipy.io.wavfile.read(path + filename)[0], scipy.io.wavfile.read(path + filename)[1], filename))
    #print(audio_list)
    return audio_list

def spg(path):
    start = True
    _spg = 0
    for filename in os.listdir(path):
        rate, data = scipy.io.wavfile.read(path + filename)
        f, t, Sxx = signal.spectrogram(data, rate, 'hamming', 512, 160, 512)
        if start:
            _spg = Sxx
            start = False
        else:
            _spg = scipy.hstack((_spg, Sxx))
    _spg = 10 * np.log10(_spg + sys.float_info.min)
    return _spg

def cspg(path1, path2):
    _spg1 = spg(path1)
    _spg2 = spg(path2)
    _spg = scipy.hstack((_spg1, _spg2))
    return _spg

def spg_b(s, U):
    ssum = np.sum(s)
    approx_sum = 0
    i = 0
    for e_value in s:
        approx_sum = approx_sum + e_value
        if approx_sum >= ssum * 0.95:
            break
        i = i + 1
    base = U[:, range(0, 20)]
    return base

#unknowns = load_audio(path_unknowns)
music = load_audio(path_music)
speech = load_audio(path_speech)

C:\Users\Mauro\PycharmProjects\mbmcpym.conte/05_AudioClassifier_Pdf/05_AudioClassifier_Pdf/database/music/


Data $M\in \mathbb{C}^{mxn}$ allora $M = U \Sigma V^*$ con $U\in \mathbb{C}^{mxm}$ , $\Sigma\in \mathbb{C}^{mxn}$ , $V\in\mathbb{C}^{nxn}$

Date $M_A$ e $M_B$ con $M_A = M_B^T$ allora $M_A = U_A \Sigma_A V_A^*$ e $M_B = U_B \Sigma_B V_B^*$ 

$M_A = M_B^T$ quindi $U_A \Sigma_A V_A^* = (U_B \Sigma_B V_B^*)^T = V_B^\dagger \Sigma_B^T U_B^T $
$$ U_A = V_B^\dagger $$
$$ \Sigma_A = \Sigma_B^T $$
$$ V_A = U_B^T$$ 

Nota1::$M^\dagger = M^*T=M^T*$

Nota2::$(ABC ... XYZ)^T = Z^T Y^T X^T ... C^T A^T B^T$

Nota3::$\mathbb{C}^{mxn}$ sono matrici a valori complessi di dimensioni mxn

In [6]:
spg_music = spg(path_music)
#print("spg_music shape: ",spg_music.shape)
U, s, V= np.linalg.svd(spg_music)
#le colonne di U sono autovettori
b_music = spg_b(s,U)#riduco la base
#ottengo una base con i vettori per colonna
print("b_music shape",b_music.shape)
#norm_music = np.dot(b_music.conj().T,b_music)
#norm_music[norm_music<0.1]=0
#norm_music[norm_music>0.9]=1
#print(norm_music)
#print("norm_music shape",norm_music.shape)

b_music shape (257, 20)


In [4]:
spg_speech = spg(path_speech)
#print("spg_speech shape: ",spg_speech.shape)
U, s, V= np.linalg.svd(spg_speech)
b_speech = spg_b(s,U)
print("b_speech shape",b_speech.shape)
#norm_speech = np.dot(b_speech.conj().T,b_speech)
#print("norm_speech shape",norm_speech.shape)

#norm_speech[norm_speech<0.1]=0
#norm_speech[norm_speech>0.9]=1

b_speech shape (257, 20)


In [5]:
spg_total = cspg(path_speech,path_music)
#print("spg_speech shape: ",spg_speech.shape)
U, s, V= np.linalg.svd(spg_total)
b_total = spg_b(s,U)
print("b_total shape",b_total.shape)
#norm_speech = np.dot(b_speech.conj().T,b_speech)
#print("norm_speech shape",norm_speech.shape)

#norm_speech[norm_speech<0.1]=0
#norm_speech[norm_speech>0.9]=1

b_total shape (257, 20)


In [6]:
for rate,data,filename in unknowns:
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    Sxx = 10*np.log10(Sxx+sys.float_info.min)
    spg_unk = Sxx
    print('Item: ',filename)
    #print('\tOnSpeech: ',Sxx.T.shape,'X',b_speech.shape,'=',np.dot(Sxx.T,b_speech).shape) 
    #print('\tOnMusic: ',Sxx.T.shape,'X',b_music.shape,'=',np.dot(Sxx.T,b_music).shape) 
    
    unk_on_music = np.dot(Sxx.T,b_music) #sono i coeff
    unk_on_speech = np.dot(Sxx.T,b_speech) #sono i coeff
    
    spg_ric_on_music = np.dot(unk_on_music,b_music.T).T
    #print('Stima sulla musica',spg_ric_on_music.shape)
    error_on_music = np.linalg.norm(spg_unk-spg_ric_on_music)
    #print('spg_unk',spg_unk.shape)
    #print('spg_ric_on_music',spg_ric_on_music.shape)
    #print('error_on_music',error_on_music.shape)
    #error_on_music = np.sum(np.power(spg_unk-spg_ric_on_music,2))
    
    spg_ric_on_speech = np.dot(unk_on_speech,b_speech.T).T
    #print('Stima sull parlato',spg_ric_on_speech.shape)
    error_on_speech = np.linalg.norm(spg_unk-spg_ric_on_speech)
    #error_on_speech = np.sum(np.power(spg_unk-spg_ric_on_speech,2))
    
    #print('\terr on music',error_on_music)
    #print('\terr on speech',error_on_speech)
    if error_on_music<error_on_speech: print('\t\t\t\tMUSICA')
    else: print('\t\t\t\tPARLATO')
    
#....considerazioni

Item:  004_BobDylan-OxfordTown-10s-B.wav
				PARLATO
Item:  007si1079.wav
				PARLATO
Item:  007si1271.wav
				PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
				MUSICA
Item:  014si1291.wav
				PARLATO
Item:  016si1621.wav
				PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
				MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
				MUSICA


In [7]:
media = np.zeros(b_music.shape[1])
i = 0
for rate,data,filename in music:
    i+=1
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    Sxx = 10*np.log10(Sxx+sys.float_info.min)
    spg_unk = Sxx
    
    unk_on_music = np.dot(Sxx.T,b_music)
    media += np.mean(unk_on_music,0)

media_m = media/i


media = np.zeros(b_speech.shape[1])
i = 0
for rate,data,filename in speech:
    i+=1
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    Sxx = 10*np.log10(Sxx+sys.float_info.min)
    spg_unk = Sxx
    
    unk_on_speech = np.dot(Sxx.T,b_speech)
    media += np.mean(unk_on_speech,0)

media_s = media/i
    
#print('mean speech',media_s)
#print('mean music',media_m)

In [8]:
for rate,data,filename in unknowns:
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    Sxx = 10*np.log10(Sxx+sys.float_info.min)
    spg_unk = Sxx
    print('Item: ',filename)
    
    unk_on_music = np.dot(Sxx.T,b_music)
    unk_on_speech = np.dot(Sxx.T,b_speech)
    
    mat_med_m = np.matrix(np.tile(media_m,[unk_on_music.shape[0],1]))
    mat_med_s = np.matrix(np.tile(media_s,[unk_on_speech.shape[0],1]))
    
    v_m = np.linalg.norm(unk_on_music-mat_med_m)
    v_s = np.linalg.norm(unk_on_speech-mat_med_s)
    
    if v_m<v_s: print('\t\t\t\tMUSICA')
    else: print('\t\t\t\tPARLATO')
    
#BELLISSIMO

Item:  004_BobDylan-OxfordTown-10s-B.wav
				MUSICA
Item:  007si1079.wav
				PARLATO
Item:  007si1271.wav
				PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
				MUSICA
Item:  014si1291.wav
				PARLATO
Item:  016si1621.wav
				PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
				MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
				MUSICA


In [9]:
media = np.zeros(b_total.shape[1])
i = 0
for rate,data,filename in music:
    i+=1
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    Sxx = 10*np.log10(Sxx+sys.float_info.min)
    spg_unk = Sxx
    
    unk_on_music = np.dot(Sxx.T,b_total)
    media += np.mean(unk_on_music,0)

media_m_t = media/i


media = np.zeros(b_total.shape[1])
i = 0
for rate,data,filename in speech:
    i+=1
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    Sxx = 10*np.log10(Sxx+sys.float_info.min)
    spg_unk = Sxx
    
    unk_on_speech = np.dot(Sxx.T,b_total)
    media += np.mean(unk_on_speech,0)

media_s_t = media/i
    
#print('mean speech',media_s_t)
#print('mean music',media_m_t)

In [10]:
#media con base unica
for rate,data,filename in unknowns:
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    Sxx = 10*np.log10(Sxx+sys.float_info.min)
    spg_unk = Sxx
    print('Item: ',filename)
    
    unk_on_base = np.dot(Sxx.T,b_total)
    
    mat_med_m_t = np.matrix(np.tile(media_m_t,[unk_on_base.shape[0],1]))
    mat_med_s_t = np.matrix(np.tile(media_s_t,[unk_on_base.shape[0],1]))
    
    v_m = np.linalg.norm(unk_on_base-mat_med_m_t)
    v_s = np.linalg.norm(unk_on_base-mat_med_s_t)
    
    if v_m<v_s: print('\t\t\t\tMUSICA')
    else: print('\t\t\t\tPARLATO')
    
#BELLISSIMO

Item:  004_BobDylan-OxfordTown-10s-B.wav
				MUSICA
Item:  007si1079.wav
				PARLATO
Item:  007si1271.wav
				PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
				MUSICA
Item:  014si1291.wav
				PARLATO
Item:  016si1621.wav
				PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
				MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
				MUSICA
